In [1]:
import pandas as pd
import numpy as np
import pickle_funcs as pk
from datetime import datetime as dt

# Running the Data Query

I'm getting this data from the [NHTSA](https://www-fars.nhtsa.dot.gov/QueryTool/QuerySection/SelectYear.aspx)

I could just do a massive data dump, but I'm not sure this makes sense. It seems better to build on columns as needed. 

Apparently it isn't possible to do multiyear queries? This makes time series annoying but that isn't too bad (are time series really central to this sort of analysis?)

I realized that the case numbers are not unique, which means that it would be hard to append in new columns. May be able to match the case number, state number, and some other identifier if needing to append a new column, but I don't think I'm going to do this at the moment. 

So I can just run another query if I want to add columns.

Process for pulling more data:

* Select a year from the dropdown in the above link
* Option 1 in the second screen probably makes the most sense 
* Select desired fields from either table 
* Select "Case Listing"
* Make sure that Case Number and State are selected for the identifier fields
* Select output stats
* Name the export
* Preview data
* Export via txt (gets a tab seperated value)  



# Creating a DataFrame

In [3]:
# Read table with date and time columns as a string and index col set to first col
data = pd.read_table('data.txt', dtype={'accdate':str, 'acctime':str}, index_col=0)
data.head()

,statenum,casenum,accdate,acctime,speeding,Unnamed: 6
Obs.,,,,,,
1,9,108,09082015,0437,0,NaN
2,13,86,01272015,1405,0,NaN
3,45,898,12262015,0115,0,NaN
4,6,2100,11052015,1110,1,NaN
5,23,41,06072015,1354,0,NaN


In [4]:
del data['Unnamed: 6'] # there are tabs at the end of these files, just have to delete a column

In [5]:
data.head()

,statenum,casenum,accdate,acctime,speeding
Obs.,,,,,
1,9,108,09082015,0437,0
2,13,86,01272015,1405,0
3,45,898,12262015,0115,0
4,6,2100,11052015,1110,1
5,23,41,06072015,1354,0


In [6]:
type(data)

pandas.core.frame.DataFrame

In [17]:
data.dtypes

statenum     int64
casenum      int64
accdate     object
acctime     object
speeding     int64
dtype: object

# Convert Dates to Python Datetime Object

In [8]:

sample_date = '09022015'
sample_time = '1617'

def convert_date(date_string, time_string):
    """Takes time and date as string and converts to datetime object"""
    string = '%s : %s' % (date_string, time_string)
    return dt.strptime(string, '%m%d%Y : %H%M')
    
    
convert_date(sample_date, sample_time)

datetime.datetime(2015, 9, 2, 16, 17)

In [9]:
data.head()

,statenum,casenum,accdate,acctime,speeding
Obs.,,,,,
1,9,108,09082015,0437,0
2,13,86,01272015,1405,0
3,45,898,12262015,0115,0
4,6,2100,11052015,1110,1
5,23,41,06072015,1354,0


In [10]:
len(data.index)

32166

In [11]:
count = 0
for item in data['acctime']:
    if item == '9999':
        count += 1
        
count

249

In [12]:
print('the new length should be:', 32166 - 249)

the new length should be: 31917


In [13]:
data = data.drop(data[data.acctime == '9999'].index)

In [14]:
len(data.index)

31917

In [15]:
# less than ten of the time data points have a '99' on the end
# these are replaced by setting the hour to '00'
for row in range(len(data.index)):
    time = data.iloc[row, 3]
    if int(time[2]) > 5:
        time = time[:2] + '00'
        data.iloc[row, 3] = time

In [16]:
# the dates are all the same length
# the times are all the same length

# the dates are all the same length
# the times are all the same length


for row in range(len(data.index)):
    date = data.iloc[row, 2]
    time = data.iloc[row, 3]

    datetime = convert_date(date, time)
    data.iloc[row, 2] = datetime



# for row in range(len(data.index)):
#     date = data.iloc[row, 2]
#     time = data.iloc[row, 3]
#     try:
#         datetime = convert_date(date, time)
#         data.iloc[row, 2] = datetime
#     except ValueError:
#         print(date)
#         print(time)



# Adding a Weekday Record

In [35]:
data.head()

,statenum,casenum,accdate,acctime,speeding,weekday
Obs.,,,,,,
1,9,108,2015-09-08 04:37:00,0437,0,1
2,13,86,2015-01-27 14:05:00,1405,0,1
3,45,898,2015-12-26 01:15:00,0115,0,5
4,6,2100,2015-11-05 11:10:00,1110,1,3
5,23,41,2015-06-07 13:54:00,1354,0,6


In [21]:
empty = []

for i in range(len(data.index)):
    empty.append('NA')
data['weekday'] = empty

In [25]:

for row in range(len(data.index)):
    weekday = data.iloc[row, 2].weekday()
    data.iloc[row, 5] = weekday


In [31]:
data.dtypes

statenum     int64
casenum      int64
accdate     object
acctime     object
speeding     int64
weekday     object
dtype: object

# Issue with too Many Records (don't run)

For some reason there are about 54K records, but this isn't correct the way that I understand. According to other sources, there were only about 30K fatalities in the US in 2015. 

I think the issue is that I was accessing data from two tables in the query (there is a 'person' table and an 'accident' table). If I want to incorporate data from the persons table, I would need to figure out how to marry these two tables. Could possibly look for data where the statenum, casenum, accdate and acctime are equal....

In [46]:
data.head() # if this table has drugres1 in it, it is from the old dataset 

,statenum,casenum,accdate,acctime,speeding,alcres,drugres1
Obs.,,,,,,,
1,6,1397,2015-09-02 16:17:00,1617,1,996,0
2,6,1734,2015-08-01 08:52:00,0852,0,996,0
3,27,263,2015-09-21 12:25:00,1225,0,996,0
4,30,113,2015-08-10 00:21:00,0021,1,262,605
5,37,47,2015-01-02 19:05:00,1905,0,0,0


In [47]:
len(data['casenum'])

54228

In [49]:
len(data['casenum'].unique())

3191

# Write Object to File

In [32]:
# Write this to a pickle file, creating the main data file. 
# Can't test to see if the dataframes are exactly the same because of issues with Pandas

pk.pickle_object(data, 'data', test=False)

In [33]:
# Load the pickled dataframe to test
new_data = pk.unpickle_object('data')

In [34]:
# vector operation on dataframes, return array with True in cell if don't match
# This cell should return a zero
test = new_data.values != data.values
test.sum()

0